# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
!pip install gmaps

In [ ]:
!jupyter nbextension enable --py --sys-prefix gmaps
!jupyter nbextension enable --py --sys-prefix widgetsnbextension


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath = '../output_data/weather_data.csv'
city_weather = pd.read_csv(csvpath)
city_weather

,City,Latitude,Longitude,Temperature,% Humidity,% Cloudiness,Wind Speed
0,Manokwari,-0.8667,134.0833,299.35,75,100,1.07
1,Saint Paul Harbor,57.7900,-152.4072,285.88,67,20,7.72
2,Bhag,29.0415,67.8239,302.17,20,0,1.91
3,Touros,-5.1989,-35.4608,299.55,76,8,6.95
4,Hermanus,-34.4187,19.2345,295.26,56,100,2.81
...,...,...,...,...,...,...,...
539,Punta Cardon,11.6581,-70.2150,303.18,67,12,10.80
540,Gunjur,13.2019,-16.7339,300.19,83,20,0.00
541,Kodinsk,58.6881,99.1844,278.54,90,96,2.70
542,Taylor,32.3001,-99.8337,296.85,64,100,2.57


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

coordinates = city_weather[['Latitude', 'Longitude']]
humidity = city_weather['% Humidity']
markers = gmaps.marker_layer(coordinates)
fig.add_layer(gmaps.heatmap_layer(coordinates, humidity, max_intensity = 200))

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
vacation_destinations = city_weather[city_weather['% Humidity'] < 20]
vacation_destinations = vacation_destinations[vacation_destinations['Temperature'] < 300]
hotel_df = vacation_destinations[vacation_destinations['Wind Speed'] < 5]
hotel_df

,City,Latitude,Longitude,Temperature,% Humidity,% Cloudiness,Wind Speed
48,Yumen,40.2833,97.2000,279.28,17,45,1.11
101,Okakarara,-20.5833,17.4333,294.96,14,5,3.17
156,Alice Springs,-23.7000,133.8833,293.90,18,89,1.54
176,Yinchuan,38.4681,106.2731,281.79,16,0,2.17
346,Riyadh,24.6877,46.7219,298.23,12,2,2.05
353,Santiago del Estero,-27.7951,-64.2615,297.85,16,0,4.20
502,Chivilcoy,-34.8957,-60.0167,296.02,13,96,4.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [60]:
hotel_df['Hotel Name'] = ''
hotel_df

C:\Users\jacke\AppData\Local\Temp\ipykernel_2000\2419251247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Latitude,Longitude,Temperature,% Humidity,% Cloudiness,Wind Speed,Hotel Name
48,Yumen,40.2833,97.2000,279.28,17,45,1.11,
101,Okakarara,-20.5833,17.4333,294.96,14,5,3.17,
156,Alice Springs,-23.7000,133.8833,293.90,18,89,1.54,
176,Yinchuan,38.4681,106.2731,281.79,16,0,2.17,
346,Riyadh,24.6877,46.7219,298.23,12,2,2.05,
353,Santiago del Estero,-27.7951,-64.2615,297.85,16,0,4.20,
502,Chivilcoy,-34.8957,-60.0167,296.02,13,96,4.36,


In [81]:
# couldn't get this to work

hotel_list = []

for city in hotel_df['City']:
    target_coordinates = hotel_df[['Latitude', 'Longitude']]
    target_search = "hotel"
    target_radius = 5000
    target_type = "hotel"

    params = {
        "location": target_coordinates,
        "search": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    response = requests.get(base_url, params=params)

    places_data = response.json()

    hotel_list.append(places_data['results'][4]['name'])

IndexError: list index out of range

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotels)
fig.add_layer(markers)

# Display figure
fig